In [1]:
import tensorflow as tf
import numpy as np
from custom_layers import *
from worker import Worker

In [2]:
coordinator = tf.train.Coordinator()

hidden_layer_size = 128
output_size = 10
global_model = FullyConnectedModel(
                output_size=output_size,
                hidden_layer_size=hidden_layer_size,
            )
#_ = global_model([global_model.inputs['img'], global_model.inputs['mask']])
#print(global_model.accumulators)
fake_img = np.zeros((1, 28, 28))
fake_mask = np.zeros((1, hidden_layer_size))
print("Init call: ", global_model([fake_img, fake_mask]))
'''
for layer in global_model.layers:
    layer.trainable = False
'''
global_model.summary()
print("weights: ", global_model.get_weights())
print("accumulators: ", global_model.accumulators)
global_model.init_accumulators()
print("accumulators: ", global_model.accumulators)
print("weights: ", global_model.get_weights())
print("trainable: ", global_model.non_trainable_weights)
#global_model.copy_weights(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Init call:  tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)
Model: "fully_connected_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_flatte multiple                  0         
_________________________________________________________________
fully_connected_model_encode multiple                  100480    
____________________________________________________________

In [3]:
global_model.trainable_weights

[<tf.Variable 'fully_connected_model/fully_connected_model_encoder/kernel:0' shape=(784, 128) dtype=float32, numpy=
 array([[-0.02339377,  0.06639843, -0.05461689, ...,  0.02317664,
         -0.07659809,  0.05744082],
        [-0.03768495,  0.03638853,  0.0432763 , ...,  0.00583328,
          0.05248032, -0.02702651],
        [ 0.01937802,  0.04064792, -0.06513675, ..., -0.02363941,
          0.06750212, -0.04918022],
        ...,
        [ 0.04492291,  0.00511966,  0.0586156 , ..., -0.0600876 ,
         -0.02894507, -0.02844824],
        [-0.05871632,  0.02465209, -0.02257762, ...,  0.02961814,
          0.07107253,  0.0323651 ],
        [ 0.00103039,  0.08030657,  0.01783152, ...,  0.03497523,
         -0.02848938, -0.05953145]], dtype=float32)>,
 <tf.Variable 'fully_connected_model/fully_connected_model_encoder/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [4]:
num_models = 4
models = {}
for idx in range(num_models):
    model = FullyConnectedModel(
            output_size=output_size,
            hidden_layer_size=hidden_layer_size,
            #parent_model=global_model,
        )
    #_ = model([model.inputs['img'], model.inputs['mask']])
    print(model([fake_img, fake_mask]))
    models[idx] = model
    #model.set_parent_model(global_model)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 10), dtype=float32)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting

In [5]:
for idx in range(num_models):
    t = Worker(args=idx, kwargs=
                {
                    'coordinator': coordinator,
                    'global_model': global_model,
                    'model': models[idx],
                    'hidden_layer_size': hidden_layer_size,
                    'output_size': output_size,
                    'learning_rate': 1e-3,
                    'dropout_rate': 1-0.5,
                    'batch_size': 64,
                    'epochs': 20,
                    'update_batches': 1,
                    'regenerate_masks': False,
                    'mask_generation_batches': 1000
                }
    )
    t.start()


Model: "fully_connected_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_1_flat multiple                  0         
_________________________________________________________________
fully_connected_model_1_enco multiple                  100480    
_________________________________________________________________
fully_connected_model_1_Cust multiple                  0         
_________________________________________________________________
fully_connected_model_1_outp multiple                  1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
None
Model: "fully_connected_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fully_connected_model_2_flat multiple                  0         

Epoch: 0	Thread: 3	step 300	mean loss = tf.Tensor(0.045925427, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7318833, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 300	mean loss = tf.Tensor(0.043680135, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7379049, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 300	mean loss = tf.Tensor(0.045227773, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7249273, shape=(), dtype=float32)
Epoch: 0	Thread: 2	step 300	mean loss = tf.Tensor(0.04469078, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7318314, shape=(), dtype=float32)
Epoch: 0	Thread: 3	step 400	mean loss = tf.Tensor(0.042448062, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7524548, shape=(), dtype=float32)
Epoch: 0	Thread: 1	step 400	mean loss = tf.Tensor(0.040434644, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.7593906, shape=(), dtype=float32)
Epoch: 0	Thread: 0	step 400	mean loss = tf.Tensor(0.04185577, shape=(), dtype=float32)
	mean ac

Epoch: 1	Thread: 0	step 600	mean loss = tf.Tensor(0.030135496, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8239966, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 600	mean loss = tf.Tensor(0.02981613, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.82746994, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 700	mean loss = tf.Tensor(0.029198319, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8295983, shape=(), dtype=float32)
Epoch: 1	Thread: 3	step 700	mean loss = tf.Tensor(0.030204212, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8239052, shape=(), dtype=float32)
Epoch: 1	Thread: 0	step 700	mean loss = tf.Tensor(0.029717889, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8264514, shape=(), dtype=float32)
Epoch: 1	Thread: 2	step 700	mean loss = tf.Tensor(0.029408539, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8299321, shape=(), dtype=float32)
Epoch: 1	Thread: 1	step 800	mean loss = tf.Tensor(0.028858323, shape=(), dtype=float32)
	mean 

Start of epoch 3
Start of epoch 3
Start of epoch 3
Epoch: 3	Thread: 3	step 0	mean loss = tf.Tensor(0.027285442, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8406622, shape=(), dtype=float32)
Start of epoch 3
Epoch: 3	Thread: 1	step 0	mean loss = tf.Tensor(0.026513875, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.84547716, shape=(), dtype=float32)
Epoch: 3	Thread: 2	step 0	mean loss = tf.Tensor(0.026617844, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.84625465, shape=(), dtype=float32)
Epoch: 3	Thread: 0	step 0	mean loss = tf.Tensor(0.026865842, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8432113, shape=(), dtype=float32)
Epoch: 3	Thread: 3	step 100	mean loss = tf.Tensor(0.027121013, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8417335, shape=(), dtype=float32)
Epoch: 3	Thread: 1	step 100	mean loss = tf.Tensor(0.026361167, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.84633493, shape=(), dtype=float32)
Epoch: 3	Thread: 2	step 100	mea

Epoch: 4	Thread: 0	step 300	mean loss = tf.Tensor(0.0253663, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.85199255, shape=(), dtype=float32)
Epoch: 4	Thread: 2	step 300	mean loss = tf.Tensor(0.025203932, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.85427207, shape=(), dtype=float32)
Epoch: 4	Thread: 3	step 400	mean loss = tf.Tensor(0.025670256, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.85022813, shape=(), dtype=float32)
Epoch: 4	Thread: 1	step 400	mean loss = tf.Tensor(0.025008883, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8546058, shape=(), dtype=float32)
Epoch: 4	Thread: 0	step 400	mean loss = tf.Tensor(0.025269805, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8526748, shape=(), dtype=float32)
Epoch: 4	Thread: 2	step 400	mean loss = tf.Tensor(0.025110293, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8548919, shape=(), dtype=float32)
Epoch: 4	Thread: 3	step 500	mean loss = tf.Tensor(0.025574982, shape=(), dtype=float32)
	mean

...

Epoch: 13	Thread: 3	step 800	mean loss = tf.Tensor(0.02245386, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8704671, shape=(), dtype=float32)
Epoch: 13	Thread: 1	step 800	mean loss = tf.Tensor(0.021932414, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87369716, shape=(), dtype=float32)
Epoch: 13	Thread: 0	step 800	mean loss = tf.Tensor(0.022077596, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87284666, shape=(), dtype=float32)
Epoch: 13	Thread: 2	step 800	mean loss = tf.Tensor(0.021965833, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.874182, shape=(), dtype=float32)
Epoch: 13	Thread: 3	step 900	mean loss = tf.Tensor(0.02244018, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87055075, shape=(), dtype=float32)
Epoch: 13	Thread: 1	step 900	mean loss = tf.Tensor(0.021918153, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87377995, shape=(), dtype=float32)
Start of epoch 14
Epoch: 13	Thread: 0	step 900	mean loss = tf.Tensor(0.022064954, shape

Epoch: 15	Thread: 0	step 100	mean loss = tf.Tensor(0.021896984, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87409097, shape=(), dtype=float32)
Epoch: 15	Thread: 2	step 100	mean loss = tf.Tensor(0.021774612, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8753265, shape=(), dtype=float32)
Epoch: 15	Thread: 3	step 200	mean loss = tf.Tensor(0.022260819, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87177825, shape=(), dtype=float32)
Epoch: 15	Thread: 1	step 200	mean loss = tf.Tensor(0.021733398, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87500983, shape=(), dtype=float32)
Epoch: 15	Thread: 0	step 200	mean loss = tf.Tensor(0.021883057, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8742244, shape=(), dtype=float32)
Epoch: 15	Thread: 2	step 200	mean loss = tf.Tensor(0.021757517, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8754568, shape=(), dtype=float32)
Epoch: 15	Thread: 3	step 300	mean loss = tf.Tensor(0.022252548, shape=(), dtype=float

Epoch: 16	Thread: 3	step 500	mean loss = tf.Tensor(0.022095507, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8728126, shape=(), dtype=float32)
Epoch: 16	Thread: 1	step 500	mean loss = tf.Tensor(0.021563908, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8761068, shape=(), dtype=float32)
Epoch: 16	Thread: 0	step 500	mean loss = tf.Tensor(0.021717723, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8752893, shape=(), dtype=float32)
Epoch: 16	Thread: 2	step 500	mean loss = tf.Tensor(0.021585718, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8764495, shape=(), dtype=float32)
Epoch: 16	Thread: 3	step 600	mean loss = tf.Tensor(0.022079987, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87292784, shape=(), dtype=float32)
Epoch: 16	Thread: 1	step 600	mean loss = tf.Tensor(0.021547347, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8762419, shape=(), dtype=float32)
Epoch: 16	Thread: 0	step 600	mean loss = tf.Tensor(0.021704035, shape=(), dtype=float32

Epoch: 17	Thread: 0	step 800	mean loss = tf.Tensor(0.021558937, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8763918, shape=(), dtype=float32)
Epoch: 17	Thread: 2	step 800	mean loss = tf.Tensor(0.021425992, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87742144, shape=(), dtype=float32)
Epoch: 17	Thread: 3	step 900	mean loss = tf.Tensor(0.02193345, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8738781, shape=(), dtype=float32)
Epoch: 17	Thread: 1	step 900	mean loss = tf.Tensor(0.02139353, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87721866, shape=(), dtype=float32)
Start of epoch 18
Epoch: 17	Thread: 0	step 900	mean loss = tf.Tensor(0.021551495, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8764364, shape=(), dtype=float32)
Epoch: 18	Thread: 3	step 0	mean loss = tf.Tensor(0.021928104, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8739195, shape=(), dtype=float32)
Epoch: 17	Thread: 2	step 900	mean loss = tf.Tensor(0.021420179, shape=()

Epoch: 19	Thread: 2	step 100	mean loss = tf.Tensor(0.021296386, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87813836, shape=(), dtype=float32)
Epoch: 19	Thread: 3	step 200	mean loss = tf.Tensor(0.021804888, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8747077, shape=(), dtype=float32)
Epoch: 19	Thread: 1	step 200	mean loss = tf.Tensor(0.021265645, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87804544, shape=(), dtype=float32)
Epoch: 19	Thread: 0	step 200	mean loss = tf.Tensor(0.02142026, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8773741, shape=(), dtype=float32)
Epoch: 19	Thread: 2	step 200	mean loss = tf.Tensor(0.02128585, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.8782207, shape=(), dtype=float32)
Epoch: 19	Thread: 3	step 300	mean loss = tf.Tensor(0.021799823, shape=(), dtype=float32)
	mean accuracy = tf.Tensor(0.87471014, shape=(), dtype=float32)
Epoch: 19	Thread: 1	step 300	mean loss = tf.Tensor(0.021260723, shape=(), dtype=float32